# Exercise: Create a BERT sentiment classifier

In this exercise, you will create a BERT sentiment classifier (actually DistilBERT) using the [Hugging Face Transformers](https://huggingface.co/transformers/) library.

You will use the [IMDB movie review dataset](https://huggingface.co/datasets/imdb) to train and evaluate your model. The IMDB dataset contains movie reviews that are labeled as either positive or negative.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
# ! pip install -q "datasets==2.15.0"

In [2]:
# Import the datasets and transformers packages
# !pip install datasets
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

## Pre-process datasets

Now we are going to process our datasets by converting all the text into tokens for our models. You may ask, why isn't the text converted already? Well, different models may use different tokenizers, so by converting at train time we retain more flexibility.

In [3]:
# Replace <MASK> with your code that constructs a query to send to the LLM
!pip install transformers

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# # Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], truncation=True, padding="max_length")


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Check that we tokenized the examples properly
# assert tokenized_ds["train"][0]["input_ids"][:5] == [101, 2045, 2003, 2053, 7189]

# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

[1858, 318, 645, 8695, 379, 477, 1022, 6401, 959, 290, 4415, 5329, 475, 262, 1109, 326, 1111, 389, 1644, 2168, 546, 6590, 6741, 13, 4415, 5329, 3073, 42807, 11, 6401, 959, 3073, 6833, 13, 4415, 5329, 21528, 389, 2407, 2829, 13, 6401, 959, 338, 7110, 389, 1290, 517, 8253, 986, 6401, 959, 3073, 517, 588, 5537, 8932, 806, 11, 611, 356, 423, 284, 4136, 20594, 986, 383, 1388, 2095, 318, 4939, 290, 7650, 78, 11, 475, 423, 366, 27659, 40024, 590, 1911, 4380, 588, 284, 8996, 11, 284, 5052, 11, 284, 13446, 13, 1374, 546, 655, 13226, 30, 40473, 1517, 1165, 11, 661, 3597, 6401, 959, 3073, 1605, 475, 11, 319, 262, 584, 1021, 11, 11810, 484, 4702, 1605, 2168, 357, 10185, 737, 6674, 340, 338, 262, 3303, 11, 393, 262, 4437, 11, 475, 314, 892, 428, 2168, 318, 517, 3594, 621, 1605, 13, 2750, 262, 835, 11, 262, 10544, 389, 1107, 922, 290, 8258, 13, 383, 7205, 318, 407, 31194, 379, 477, 986, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

## Load and set up the model

We will now load the model and freeze most of the parameters of the model: everything except the classification head.

In [4]:
# Replace <MASK> with your code freezes the base model
#install pytorch 

# !pip install torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)


#use peft library to load the model
# !pip install peft
from peft import get_peft_model
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# model_name_or_path = "bigscience/mt0-large"
# tokenizer_name_or_path = "bigscience/mt0-large"

config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

lora_model = get_peft_model(model, config)

print(lora_model.print_trainable_parameters())

# Freeze all the parameters of the base model
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
# for param in model.base_model.parameters():
#     # freaze all the parameters
#     param.requires_grad = False

# model.classifier

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.2364
None


/opt/conda/lib/python3.12/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
# Replace <MASK> with your code freezes the base model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)


#use peft library to load the model
# !pip install peft
from peft import get_peft_model
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# model_name_or_path = "bigscience/mt0-large"
# tokenizer_name_or_path = "bigscience/mt0-large"

# Change the task type to CAUSAL_LM
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

lora_model = get_peft_model(model, config)

print(lora_model.print_trainable_parameters())

# Freeze all the parameters of the base model
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
# for param in model.base_model.parameters():
#     # freaze all the parameters
#     param.requires_grad = False

# model.classifier

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.2364
None


In [6]:
print(lora_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict

## Let's train it!

Now it's time to train our model. We'll use the `Trainer` class from the 🤗 Transformers library to do this. The `Trainer` class provides a high-level API that abstracts away a lot of the training loop.

First we'll define a function to compute our accuracy metreic then we make the `Trainer`.

Let's take this opportunity to learn about the `DataCollator`. According to the HuggingFace documentation:

> Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.

> To be able to build batches, data collators may apply some processing (like padding).


In [7]:
# Replace <MASK> with your DataCollatorWithPadding argument(s)
# install accelerate
# !pip install transformers[torch]
# !pip install transformers[torch]
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

lora_model.config.pad_token_id = tokenizer.pad_token_id
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    #add data_collector paprameter
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_281897/3039951264.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.086200,0.704486,0.838000


TrainOutput(global_step=500, training_loss=1.08617236328125, metrics={'train_runtime': 2447.7663, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.204, 'total_flos': 262202720256000.0, 'train_loss': 1.08617236328125, 'epoch': 1.0})

## Evaluate the model

Evaluating the model is as simple as calling the evaluate method on the trainer object. This will run the model on the test set and compute the metrics we specified in the compute_metrics function.

In [8]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 0.7044864296913147,
 'eval_accuracy': 0.838,
 'eval_runtime': 452.8074,
 'eval_samples_per_second': 1.104,
 'eval_steps_per_second': 1.104,
 'epoch': 1.0}

### View the results

Let's look at two examples with labels and predicted values.

In [9]:
import pandas as pd

df = pd.DataFrame(tokenized_ds["test"])
df = df[["text", "label"]]

# Replace <br /> tags in the text with spaces
df["text"] = df["text"].str.replace("<br />", " ")

# Add the model predictions to the dataframe
predictions = trainer.predict(tokenized_ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head(2)

,text,label,predicted_label
0,When I unsuspectedly rented A Thousand Acres...,1,1
1,This is the latest entry in the long series of...,1,1


### Look at some of the incorrect predictions

Let's take a look at some of the incorrectly-predcted examples

In [10]:
# Show full cell output
pd.set_option("display.max_colwidth", None)

df[df["label"] != df["predicted_label"]].head(2)

,text,label,predicted_label
21,"Coming from Kiarostami, this art-house visual and sound exposition is a surprise. For a director known for his narratives and keen observation of humans, especially children, this excursion into minimalist cinematography begs for questions: Why did he do it? Was it to keep him busy during a vacation at the shore? ""Five, 5 Long Takes"" consists of, you guessed it, five long takes. They are (the title names are my own and the times approximate): ""Driftwood and waves"". The camera stands nearly still looking at a small piece of driftwood as it gets moved around by small waves splashing on a beach. Ten minutes. ""Watching people on the boardwalk"". The camera stands still looking at the ocean horizon and a boardwalk. People walk across the camera frame, their faces too far and blurry to make them interesting. Eleven minutes. ""Six dogs at the water's edge"". The camera stands still looking at the ocean horizon with a sandy stretch of beach nearby. Far away at the water's edge, six dogs not doing much, just relaxing. Sixteen minutes. ""Ducks in line, gaggle of ducks"". The camera stands still looking at the ocean horizon near the water's edge. Dozen and dozen of ducks stream in single file from left to right. I assume that Kiarostami released them gradually. The last two ducks stop dead on their track and suddenly a gaggle of ducks rolls quietly from right to left. I assume Kiarostami collected the ducks and re-released all at the same time. It is not the first time that he deals with the contrast between organized and disorganized behavior. Eight minutes. ""Frog symphony, oops, I mean cacophony, for a stormy night"". The camera stands over a pond at night. It's pitch black except for what appears to be the reflection of the moon on the undulating water. It is a stormy night and clouds race to cover the moon. The screen goes dark. What remains for us is the cacophony of frogs, howling dogs and, eventually, morning roosters. Hit me on the head if this was done in a single take. I saw this segment as a sound composition put together in the editing room and accompanied by a simple visualization. Twenty seven minutes! Except for the mildly amusing ducks, this exercise in minimalism left me cold. A nonessential film for Kiarostami admirers. I thought I would rate ""Five"" a five, but four is what it deserves. The film is dedicated to Yasujiru Ozu.",0,1
27,"It could be easy to complain about the quality of this movie (you don't have to throw cartloads of money at a movie to make it good, nor will it guarantee that it is worth watching) but I think that is totally missing the point. If your expecting fast cars, T&A or a movie that will spell itself out for you then don't watch this, you'll be disappointed and dumbfounded. This movie was thoroughly enjoyable, kept us on the edge of our seats and made us really think. The writer obviously put a lot of thought and research behind this movie and it shows through the end, just remember to keep an open mind. Note: the school scenes were all filmed at McMaster University and most of the rest was done in Toronto.",1,0


## End of exercise

Great work! 🤗